In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'95c5yzw5'

In [3]:
wandb.init(id='95c5yzw5', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13503.621

In [5]:
sum([torch.numel(p) for p in m.backbone.parameters()]) / 1000

10696.232

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
m

EffNet(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True

In [8]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [9]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [10]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.68 µs


In [11]:
#wandb.watch(m, log="all")

In [12]:
#train_only_parts_of_model(m, part_to_train="_fcs_2")

In [13]:
model_stem = "8.9"
START_E = 0

In [14]:
n_epochs = 1000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 6e-4 if e < 10 else 3e-4 if e < 20 else 1e-4
    #opt.param_groups[0]['lr'] = 3e-4 if e < 10 else 1e-4
    
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.01678079, 'trn_headings_loss': 0.00085058, 'trn_curvatures_loss': 0.00285202, 'consistency losses/trn_steer_cost': 0.02032351, 'consistency losses/trn_te_loss': 20.41033395, 'aux losses/trn_pitch_loss': 0.26197958, 'aux losses/trn_yaw_loss': 0.3631506, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8984375, 'consistency losses/trn_torque_loss': 882.27319479, 'consistency losses/trn_torque_delta_loss': 17.54271979, 'logistical/obs_consumed_per_second': 232.48031496, 'logistical/obs_generated_per_second': 110.75590551, 'logistical/data_consumption_ratio': 2.0991818, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.1793499, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 232.28125, 'logistical/obs_generated_per_second': 106.4609375, 'logistical/data_consumption_ratio': 2.18542955, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01808476, 'trn_headings_loss': 0.00098618, 'trn_curvatures_loss': 0.00323085

{'logistical/obs_consumed_per_second': 229.5703125, 'logistical/obs_generated_per_second': 97.8125, 'logistical/data_consumption_ratio': 2.35237955, 'logistical/manual_train_pause': 0.00078125, 'trn_control_loss': 0.04643148, 'trn_headings_loss': 0.00184057, 'trn_curvatures_loss': 0.00647113, 'consistency losses/trn_steer_cost': 0.02718061, 'consistency losses/trn_te_loss': 19.2930225, 'aux losses/trn_pitch_loss': 0.31421232, 'aux losses/trn_yaw_loss': 0.48877335, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9375, 'consistency losses/trn_torque_loss': 848.0393289, 'consistency losses/trn_torque_delta_loss': 19.02898339, 'logistical/max_param': 16.21743202, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 239.1640625, 'logistical/obs_generated_per_second': 109.640625, 'logistical/data_consumption_ratio': 2.18271069, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02398038, 'trn_headings_loss': 0.00112278, 'trn_curvatures_loss': 0.00362621, 'consis

{'logistical/obs_consumed_per_second': 231.140625, 'logistical/obs_generated_per_second': 111.3828125, 'logistical/data_consumption_ratio': 2.07790931, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02512169, 'trn_headings_loss': 0.00123084, 'trn_curvatures_loss': 0.00433701, 'consistency losses/trn_steer_cost': 0.02510142, 'consistency losses/trn_te_loss': 19.58527605, 'aux losses/trn_pitch_loss': 0.21405077, 'aux losses/trn_yaw_loss': 0.31900692, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.953125, 'consistency losses/trn_torque_loss': 925.41747074, 'consistency losses/trn_torque_delta_loss': 19.4137338, 'logistical/max_param': 16.20981598, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 233.3671875, 'logistical/obs_generated_per_second': 103.4765625, 'logistical/data_consumption_ratio': 2.26055167, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02361989, 'trn_headings_loss': 0.00120289, 'trn_curvatures_loss': 0.00377596, 'consis

{'logistical/obs_consumed_per_second': 227.9140625, 'logistical/obs_generated_per_second': 100.5078125, 'logistical/data_consumption_ratio': 2.27395873, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.02604097, 'trn_headings_loss': 0.00114755, 'trn_curvatures_loss': 0.00404257, 'consistency losses/trn_steer_cost': 0.0233134, 'consistency losses/trn_te_loss': 19.42780238, 'aux losses/trn_pitch_loss': 0.29975605, 'aux losses/trn_yaw_loss': 0.35273314, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9609375, 'consistency losses/trn_torque_loss': 881.9139812, 'consistency losses/trn_torque_delta_loss': 17.92466437, 'logistical/max_param': 16.20093155, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 231.96875, 'logistical/obs_generated_per_second': 102.90625, 'logistical/data_consumption_ratio': 2.25384474, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0312317, 'trn_headings_loss': 0.00137915, 'trn_curvatures_loss': 0.00463412, 'con

{'logistical/obs_consumed_per_second': 231.1484375, 'logistical/obs_generated_per_second': 101.046875, 'logistical/data_consumption_ratio': 2.28919788, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02547401, 'trn_headings_loss': 0.00121937, 'trn_curvatures_loss': 0.00435902, 'consistency losses/trn_steer_cost': 0.02798128, 'consistency losses/trn_te_loss': 19.39928116, 'aux losses/trn_pitch_loss': 0.2244854, 'aux losses/trn_yaw_loss': 0.35133505, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9375, 'consistency losses/trn_torque_loss': 952.96142006, 'consistency losses/trn_torque_delta_loss': 17.20773181, 'logistical/max_param': 16.17248917, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 237.0625, 'logistical/obs_generated_per_second': 100.4140625, 'logistical/data_consumption_ratio': 2.36084367, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.02702093, 'trn_headings_loss': 0.0012866, 'trn_curvatures_loss': 0.00484119, 'consis

...

{'logistical/obs_consumed_per_second': 228.6328125, 'logistical/obs_generated_per_second': 101.6328125, 'logistical/data_consumption_ratio': 2.24640868, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01513284, 'trn_headings_loss': 0.00070965, 'trn_curvatures_loss': 0.00253945, 'consistency losses/trn_steer_cost': 0.02442783, 'consistency losses/trn_te_loss': 20.12509713, 'aux losses/trn_pitch_loss': 0.14486313, 'aux losses/trn_yaw_loss': 0.21710205, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9375, 'consistency losses/trn_torque_loss': 818.91549802, 'consistency losses/trn_torque_delta_loss': 13.72118643, 'logistical/max_param': 16.19374466, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 230.5234375, 'logistical/obs_generated_per_second': 104.1484375, 'logistical/data_consumption_ratio': 2.22064258, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01316878, 'trn_headings_loss': 0.00064449, 'trn_curvatures_loss': 0.00223635, 'consis

{'logistical/obs_consumed_per_second': 234.1015625, 'logistical/obs_generated_per_second': 99.7421875, 'logistical/data_consumption_ratio': 2.3523362, 'logistical/manual_train_pause': 0.00046875, 'trn_control_loss': 0.01556578, 'trn_headings_loss': 0.00072541, 'trn_curvatures_loss': 0.00259245, 'consistency losses/trn_steer_cost': 0.02783561, 'consistency losses/trn_te_loss': 19.69072661, 'aux losses/trn_pitch_loss': 0.15342402, 'aux losses/trn_yaw_loss': 0.21806431, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7890625, 'consistency losses/trn_torque_loss': 722.62848544, 'consistency losses/trn_torque_delta_loss': 13.24163436, 'logistical/max_param': 16.19167519, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 237.640625, 'logistical/obs_generated_per_second': 95.7890625, 'logistical/data_consumption_ratio': 2.48079535, 'logistical/manual_train_pause': 0.00210938, 'trn_control_loss': 0.0135271, 'trn_headings_loss': 0.0006161, 'trn_curvatures_loss': 0.002260

KeyboardInterrupt: 

Error in callback <function _WandbInit._pause_backend at 0x7f66675ab310> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [15]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e33.torch")

Error in callback <function _WandbInit._resume_backend at 0x7f66675ab9d0> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7f66675ab310> (for post_run_cell):


Exception: The wandb backend process has shutdown

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 149, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 120, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 411, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface

In [17]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
